In [66]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [67]:
import tensorflow as tf

In [68]:
df = pd.read_csv('/Users/user/Documents/Spam_Detection/Data/data_clear.csv')
df = df.drop(columns='index', axis=1)
X = df['Message'].tolist()
Y = df['Category'].tolist()

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [70]:
print(X_train)

['Old Orchard near univ. How about you?', 'Thanks for your ringtone order, reference number X49. Your mobile will be charged 4.50. Should your tone not arrive please call customer services 09065989182. From: [colour=red]text[/colour]TXTstar', "What is this 'hex' place you talk of? Explain!", 'Its ok, called mom instead have fun', 'Webpage s not available!', "It's ok i noe u're busy but i'm really too bored so i msg u. I oso dunno wat colour she choose 4 me one.", 'No da if you run that it activate the full version da.', 'Ok but knackered. Just came home and went to sleep! Not good at this full time work lark.', 'I want to be there so i can kiss you and feel you next to me', 'I did. One slice and one breadstick. Lol', 'WE REGRET TO INFORM U THAT THE NHS HAS MADE A MISTAKE.U WERE NEVER ACTUALLY BORN.PLEASE REPORT 2 YOR LOCAL HOSPITAL 2B TERMINATED.WE R SORRY 4 THE INCONVENIENCE', 'REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [72]:
print(X_train)

['Old Orchard near univ. How about you?', 'Thanks for your ringtone order, reference number X49. Your mobile will be charged 4.50. Should your tone not arrive please call customer services 09065989182. From: [colour=red]text[/colour]TXTstar', "What is this 'hex' place you talk of? Explain!", 'Its ok, called mom instead have fun', 'Webpage s not available!', "It's ok i noe u're busy but i'm really too bored so i msg u. I oso dunno wat colour she choose 4 me one.", 'No da if you run that it activate the full version da.', 'Ok but knackered. Just came home and went to sleep! Not good at this full time work lark.', 'I want to be there so i can kiss you and feel you next to me', 'I did. One slice and one breadstick. Lol', 'WE REGRET TO INFORM U THAT THE NHS HAS MADE A MISTAKE.U WERE NEVER ACTUALLY BORN.PLEASE REPORT 2 YOR LOCAL HOSPITAL 2B TERMINATED.WE R SORRY 4 THE INCONVENIENCE', 'REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with

In [73]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)

In [74]:

class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item
    def __len__(self):
        return len(self.labels)

In [75]:
train_dataset = SpamDataset(train_encodings, y_train)
test_dataset = SpamDataset(test_encodings, y_test)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [77]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='no',
    logging_dir='./logs',
    learning_rate=3e-5,
)

In [78]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

C:\Users\user\AppData\Local\Temp\ipykernel_11832\3441253692.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [79]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.038036
2,0.053900,0.038992
3,0.053900,0.042248


TrainOutput(global_step=774, training_loss=0.03661389575756181, metrics={'train_runtime': 471.164, 'train_samples_per_second': 26.265, 'train_steps_per_second': 1.643, 'total_flos': 1513530929295000.0, 'train_loss': 0.03661389575756181, 'epoch': 3.0})

In [80]:
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
y_pred_labels = np.argmax(logits, axis=1)


In [81]:
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels)
recall = recall_score(y_test, y_pred_labels)
f1 = f1_score(y_test, y_pred_labels)
conf_matrix = confusion_matrix(y_test, y_pred_labels)


In [82]:
print(f"Précision : {precision:.2f}")
print(f"Rappel : {recall:.2f}")
print(f"F1-score : {f1:.2f}")
print(f"Matrice de confusion :\n{conf_matrix}")

Précision : 0.97
Rappel : 0.97
F1-score : 0.97
Matrice de confusion :
[[893   4]
 [  4 131]]
